# Layerquantizer_demo.ipynb: Demonstrate the LayerQuantizer compressor on WeatherBench(ERA5) data

This workbook demonstrates the simple primary use-case of LayerQuantizer, to compress gridded geophysical data with the WeatherBench ERA5 data set.

Model training typically requires a high-performance (usually local) mirror of the WeatherBench data, but even with the baseline level of compression offered by xarray/zarr this uses a lot of disk space.

LayerQuantizer lets us quantize the data (to a few ppm accuracy, relative the layer-wise (max-min)), which is then encoded and written to disk in a way that affords much better compression ratios.

### Installation and imports

In [1]:
%pip install git+https://github.com/csubich/layerquantizer.git xarray zarr ipython

  Cloning https://github.com/csubich/layerquantizer.git to /tmp/pip-req-build-fwm8stb8
  Running command git clone --filter=blob:none --quiet https://github.com/csubich/layerquantizer.git /tmp/pip-req-build-fwm8stb8
  Resolved https://github.com/csubich/layerquantizer.git to commit f48efe343a0c4c7fef443873f0bfdcd769650e4c
  Preparing metadata (setup.py) ... done


In [2]:
import numpy as np
import xarray as xr
import zarr
import datetime
from layerquantizer import LayerQuantizer
from IPython.display import display, Markdown

## Create LayerQuantizer object for compression, with default options

In [3]:
# Create LayerQuantizer for compressor; this can also be invoked by name with otherwise default options
lq = LayerQuantizer()
lq

LayerQuantizer(blosc_clevel=5, blosc_cname='zstd', bloscer=Blosc(cname='zstd', clevel=5, shuffle=SHUFFLE, blocksize=0), nbits=16, pow2_range=False, transform='Lorenzo')

## Open the WeatherBench ERA5 dataset and compress a random selection of dates

### Open the WeatherBench dataset

The `{'token' : 'anon'}` dictionary passed to `storage_otpions` lets the download proceed on a Google colab notebook without demanding authentication.

In [4]:
wb_url = 'gs://weatherbench2/datasets/era5/1959-2023_01_10-full_37-1h-0p25deg-chunk-1.zarr'

In [5]:
wb_era5 = xr.open_zarr(wb_url,chunks={},storage_options={'token' : 'anon'})
wb_era5

<xarray.Dataset> Size: 669TB
Dimensions:                                           (time: 561264,
                                                       latitude: 721,
                                                       longitude: 1440,
                                                       level: 37)
Coordinates:
  * time                                              (time) datetime64[ns] 4MB ...
  * latitude                                          (latitude) float32 3kB ...
  * longitude                                         (longitude) float32 6kB ...
  * level                                             (level) int64 296B 1 .....
Data variables: (12/48)
    10m_u_component_of_wind                           (time, latitude, longitude) float32 2TB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    10m_v_component_of_wind                           (time, latitude, longitude) float32 2TB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    2m_dewpoint_temperature                           (time, latitude, longitude) float32 2TB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    2m_temperature                                    (time, latitude, longitude) float32 2TB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    angle_of_sub_gridscale_orography                  (latitude, longitude) float32 4MB dask.array<chunksize=(721, 1440), meta=np.ndarray>
    anisotropy_of_sub_gridscale_orography             (latitude, longitude) float32 4MB dask.array<chunksize=(721, 1440), meta=np.ndarray>
    ...                                                ...
    v_component_of_wind                               (time, level, latitude, longitude) float32 86TB dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>
    vertical_velocity                                 (time, level, latitude, longitude) float32 86TB dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>
    volumetric_soil_water_layer_1                     (time, latitude, longitude) float32 2TB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    volumetric_soil_water_layer_2                     (time, latitude, longitude) float32 2TB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    volumetric_soil_water_layer_3                     (time, latitude, longitude) float32 2TB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    volumetric_soil_water_layer_4                     (time, latitude, longitude) float32 2TB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>

### Select a few random dates to use for compression testing

In [6]:
gen = np.random.default_rng(seed=42)
NTime = 10
seltime = np.sort(gen.choice(wb_era5.time.data,size=NTime,replace=False))

In [7]:
# Subset the ERA5 data to the selected times
data = wb_era5.sel(time=seltime)

### Zarr storage helper function

This testing code uses a Zarr `MemoryStore` object to "write" the compressed data rather than create an on-disk Zarr.  The `MemoryStore` object still allows us to see what the on-disk size of the array will be, but it does not depend on genuine disk access with all of the performance and permission uncertainty so-contained.

In [8]:
# Helper function to get the size of a Zarr storage object
async def store_size(store):
  ''' Get the total size of a provided zarr storage object by aggregating
  the size of each of its keys'''
  return sum( [await store.getsize(k) async for k in store.list()] )

### Compress each 3D (with 'evel') ERA5 variable and record statistics

Note that this takes some time, mostly because of the downloading.


In [10]:
# Compress and print a table of relative compression rates

# Accumulation variables for timings
default_write_time = 0
lq_write_time = 0
default_read_time = 0
lq_read_time = 0

total_memory_bytes = 0
total_default_bytes = 0
total_lq_bytes = 0

table = [] # Array to consolidate table entries
for v in data.data_vars:
  # Skip the variable if it is a 2D field
  if ('level' not in data[v].dims): continue

  dv = data[[v]].compute() # Effect the download for a single variable

  # Zarr storage objects to simulate the written-to-disk size
  default_store = zarr.storage.MemoryStore() # With default options
  lq_store = zarr.storage.MemoryStore() # For with-LQ compression

  raw_bytes = dv.nbytes
  total_memory_bytes += raw_bytes

  # Store to 'disk' with default settings and LQ and provide timings
  tic = datetime.datetime.now()
  dv.to_zarr(default_store,compute=True,zarr_format=2).close()
  toc = datetime.datetime.now()
  default_write_time += (toc-tic).total_seconds()

  tic = datetime.datetime.now()
  dv.to_zarr(lq_store,compute=True,encoding={v : {'compressor' : lq}},zarr_format=2).close()
  toc = datetime.datetime.now()
  lq_write_time += (toc-tic).total_seconds()

  # Reload the default-compressed values for timing only
  tic = datetime.datetime.now()
  reload = xr.open_zarr(default_store).compute()
  toc = datetime.datetime.now()
  default_read_time += (toc-tic).total_seconds()

  # Reload the LQ-compressed values to test accuracy and provide timing
  tic = datetime.datetime.now()
  reload = xr.open_zarr(lq_store).compute()
  toc = datetime.datetime.now()
  lq_read_time += (toc-tic).total_seconds()
  # Get the full-scale (max-min) difference, per time and per level
  full_scale_diff = dv.max(dim=('latitude','longitude')) - \
                    dv.min(dim=('latitude','longitude'))
  # Find the maximum difference between the quantized and raw values, relative
  # to this scale
  reldiff = np.abs(reload[v] - dv[v]).max(dim=('latitude','longitude'))
  max_err = (reldiff/full_scale_diff).max()[v].data

  default_bytes = await store_size(default_store)
  lq_bytes = await store_size(lq_store)

  lq_ratio = raw_bytes / lq_bytes
  default_ratio = raw_bytes / default_bytes

  total_default_bytes += default_bytes
  total_lq_bytes += lq_bytes

  # Append stats to a markdown table
  # columns: variable, raw size, ratio default, ratio LQ, err LQ
  table.append(f'{v} | ${raw_bytes / 2**20 : .0f}$ MiB | ${default_ratio:.1f}$ (${default_bytes / 2**20 : .0f}$ MiB) |' \
                f'${lq_ratio:.1f}$ (${lq_bytes / 2**20 : .0f}$ MiB) | ${max_err * 1e6 : .2f} \\cdot 10^{{-6}}$')



## Compression results

### Compression ratios

In [14]:
table_header = 'Variable | Memory Size | Default ratio | LQ ratio | FS err'
table_style = ':-:|:-:|:-:|:-:|:-:'
display(Markdown('\n'.join([table_header,table_style]+table)))

display(Markdown(f'Compression ratios: default ${total_memory_bytes/total_default_bytes:.1f} \\times$, ' \
                 f'LayerQuantizer ${total_memory_bytes/total_lq_bytes:.1f} \\times$'))

Variable | Memory Size | Default ratio | LQ ratio | FS err
:-:|:-:|:-:|:-:|:-:
geopotential | $ 1465$ MiB | $1.9$ ($ 766$ MiB) |$5.5$ ($ 267$ MiB) | $ 7.81 \cdot 10^{-6}$
potential_vorticity | $ 1465$ MiB | $1.4$ ($ 1049$ MiB) |$3.7$ ($ 396$ MiB) | $ 7.66 \cdot 10^{-6}$
specific_humidity | $ 1465$ MiB | $1.5$ ($ 991$ MiB) |$3.6$ ($ 407$ MiB) | $ 7.71 \cdot 10^{-6}$
temperature | $ 1465$ MiB | $1.8$ ($ 811$ MiB) |$3.8$ ($ 383$ MiB) | $ 7.78 \cdot 10^{-6}$
u_component_of_wind | $ 1465$ MiB | $1.3$ ($ 1121$ MiB) |$3.8$ ($ 388$ MiB) | $ 7.69 \cdot 10^{-6}$
v_component_of_wind | $ 1465$ MiB | $1.3$ ($ 1160$ MiB) |$3.7$ ($ 401$ MiB) | $ 7.69 \cdot 10^{-6}$
vertical_velocity | $ 1465$ MiB | $1.2$ ($ 1203$ MiB) |$3.2$ ($ 455$ MiB) | $ 7.68 \cdot 10^{-6}$

Compression ratios: default $1.4 \times$, LayerQuantizer $3.8 \times$

### Read/write performance

In [18]:
display(Markdown(f'Write rates: default @ {total_memory_bytes / default_write_time / 2**20 :.2f} MiB/sec, '\
                  f'LQ @ {total_memory_bytes / lq_write_time / 2**20 :.2f} MiB/sec'))
display(Markdown(f'Read rates: default @ {total_memory_bytes / default_read_time / 2**20 :.2f} MiB/sec, '\
                  f'LQ @ {total_memory_bytes / lq_read_time / 2**20 :.2f} MiB/sec'))

Write rates: default @ 656.77 MiB/sec, LQ @ 57.06 MiB/sec

Read rates: default @ 606.57 MiB/sec, LQ @ 210.08 MiB/sec